# Loading data

In [1]:
using JuMP
using CSV
using DataFrames
using HiGHS
using PlotlyJS
using StatsBase

In [2]:
lines = CSV.read("/Users/alexanderklaus/Desktop/Masterthesis/Code/data/lines.csv", DataFrame)
#demand = CSV.read("/Users/alexanderklaus/Desktop/Masterthesis/Code/data/demand.csv", DataFrame)
#busses = CSV.read("/Users/alexanderklaus/Desktop/Masterthesis/Code/data/busses.csv", DataFrame)
bus_line_stops = CSV.read("/Users/alexanderklaus/Desktop/Masterthesis/Code/data/bus-lines.csv", DataFrame)

Row,bus_line_id,stop_ids,stop_x,stop_y
,Int64,Int64,Float64,Float64
1,1,1,2.2,36.8
2,1,2,4.4,40.1
3,1,3,12.5,42.7
4,1,4,20.2,44.0
5,2,1,40.9,48.2
6,2,2,46.2,40.7
7,2,3,56.6,38.2
8,3,1,44.6,15.2
9,3,2,52.0,20.4


In [3]:
lines

Row,line_id,bus_line_id,start_time
,Int64,Int64,Float64
1,1,1,20.0
2,2,1,40.0
3,3,1,60.0
4,1,2,20.0
5,2,2,35.0
6,3,2,50.0
7,1,3,16.0
8,2,3,32.0
9,3,3,48.0


# Data understanding

## Adding the depot

In [4]:
depot=(bus_line_id=0,stop_ids=0,stop_x=30,stop_y=30)
insert!(bus_line_stops,1,depot)
depot=(line_id=0,bus_line_id=0, start_time=0)
insert!(lines,1,depot)

Row,line_id,bus_line_id,start_time
,Int64,Int64,Float64
1,0,0,0.0
2,1,1,20.0
3,2,1,40.0
4,3,1,60.0
5,1,2,20.0
6,2,2,35.0
7,3,2,50.0
8,1,3,16.0
9,2,3,32.0


## Looking at the stops

In [37]:
using PlotlyJS
using DataFrames
using Statistics

# Gruppiere nach bus_line_id
grouped_data = groupby(bus_line_stops, :bus_line_id)

# Sammle alle Traces
traces = PlotlyJS.AbstractTrace[]  # use correct type

# Linien- und Beschriftungstraces erzeugen
for grp in grouped_data
    sort!(grp, :stop_ids)

    # Linien-Trace mit Markern
    push!(traces, scatter(
        x = grp.stop_x,
        y = grp.stop_y,
        mode = "lines+markers+text",
        name = "Linie $(unique(grp.bus_line_id)[1])",
        marker = attr(size = 8),
        text = string.(grp.stop_ids),
        textposition = "top center",
        textfont = attr(color="black", size=10)
    ))

    # Distanzen zwischen benachbarten Haltestellen
    for i in 1:(nrow(grp) - 1)
        x1, y1 = grp.stop_x[i], grp.stop_y[i]
        x2, y2 = grp.stop_x[i+1], grp.stop_y[i+1]
        dist = round(sqrt((x2 - x1)^2 + (y2 - y1)^2), digits=2)
        mx, my = mean([x1, x2]), mean([y1, y2])

        # Distanztext hinzufügen
        push!(traces, scatter(
            x = [mx], y = [my],
            mode = "text",
            text = ["d = $dist"],
            textposition = "bottom center",
            textfont = attr(size = 9, color = "gray"),
            showlegend = false
        ))
    end
end

# Layout festlegen
layout = Layout(
    title = "Buslinien und ihre Haltestellen",
    xaxis_title = "X-Koordinate",
    yaxis_title = "Y-Koordinate",
    legend = attr(x=1.02, y=1)
)

# Plot erstellen
p = Plot(traces, layout)

# In HTML-Datei speichern (im gleichen Ordner wie Code)
PlotlyJS.savefig(p, "/Users/alexanderklaus/Desktop/Masterthesis/Code/output/buslinien_plot.html")


"buslinien_plot.html"

# Preprocessing

### Input settings

In [512]:
bus_velocity = 50 #in km/h

50

### Creating nodes for all stops, on every busline, for every tour

In [513]:
nodes = []

for r in eachrow(lines)
    for row in eachrow(bus_line_stops)
        if r.bus_line_id == row.bus_line_id
            start_time_temp=0.0
            if row.stop_ids == 1
                start_time_temp = r.start_time
            else
                # alle Stopps dieser Linie sortieren
                stops = sort(filter(r2 -> r2.bus_line_id == row.bus_line_id, eachrow(bus_line_stops)), by = r2 -> r2.stop_ids)
                
                # Travel time vom ersten Knoten bis aktuellen Knoten:
                total_distance = 0.0
                for k in 1:(row.stop_ids - 1)
                    x1, y1 = stops[k].stop_x, stops[k].stop_y
                    x2, y2 = stops[k+1].stop_x, stops[k+1].stop_y
                    total_distance += sqrt((x2 - x1)^2 + (y2 - y1)^2)
                end
                travel_time = (total_distance / bus_velocity) * 60  # Minuten

                # Startzeit = Startzeit erster Knoten + Travel Time
                start_time_temp = r.start_time + travel_time
            end
            push!(nodes, (
                bus_line_id = row.bus_line_id,
                line_id     = r.line_id,
                stop_id     = row.stop_ids,
                coord_x     = row.stop_x,
                coord_y     = row.stop_y,
                start_time  = start_time_temp
            ))
                
        end
    end
end

In [514]:
print("Nodes:")
nodes

Nodes:

68-element Vector{Any}:
 (bus_line_id = 0, line_id = 0, stop_id = 0, coord_x = 30.0, coord_y = 30.0, start_time = 0.0)
 (bus_line_id = 1, line_id = 1, stop_id = 1, coord_x = 2.2, coord_y = 36.8, start_time = 20.0)
 (bus_line_id = 1, line_id = 1, stop_id = 2, coord_x = 4.4, coord_y = 40.1, start_time = 24.75932768361247)
 (bus_line_id = 1, line_id = 1, stop_id = 3, coord_x = 12.5, coord_y = 42.7, start_time = 34.967794757566324)
 (bus_line_id = 1, line_id = 1, stop_id = 4, coord_x = 20.2, coord_y = 44.0, start_time = 44.338557800136556)
 (bus_line_id = 1, line_id = 2, stop_id = 1, coord_x = 2.2, coord_y = 36.8, start_time = 40.0)
 (bus_line_id = 1, line_id = 2, stop_id = 2, coord_x = 4.4, coord_y = 40.1, start_time = 44.75932768361247)
 (bus_line_id = 1, line_id = 2, stop_id = 3, coord_x = 12.5, coord_y = 42.7, start_time = 54.967794757566324)
 (bus_line_id = 1, line_id = 2, stop_id = 4, coord_x = 20.2, coord_y = 44.0, start_time = 64.33855780013656)
 (bus_line_id = 1, line_id = 3, stop

### Creating connections

In [515]:
function eucleadian_distance(x1, y1, x2, y2)
    return sqrt((x1 - x2)^2 + (y1 - y2)^2)
end

eucleadian_distance (generic function with 1 method)

In [516]:
# Define a struct to hold the travel_time and end_time
struct Connection
    distance::Float64
    travel_time::Float64
    end_time::Float64
end

# Update the dictionary to use the struct as the value type
connections_dict = Dict()

for node1 in nodes
    for node2 in nodes
        if node1 != node2
            #skip if the nodes are the same
            origin_node = (node1.bus_line_id, node1.line_id, node1.stop_id)
            goal_node = (node2.bus_line_id, node2.line_id, node2.stop_id)
            
            if node1.bus_line_id == node2.bus_line_id
                if node1.line_id == node2.line_id
                    if node1.stop_id == node2.stop_id
                        # Nodes are the same
                        break
                    else
                        # Nodes are on the same tour
                        stops = sort(filter(n -> n.bus_line_id == node1.bus_line_id, nodes), by = n -> n.stop_id)
                        start_index = findfirst(n -> n.stop_id == node1.stop_id, stops)
                        end_index = findfirst(n -> n.stop_id == node2.stop_id, stops)
                        
                        if start_index < end_index
                            #if origin node comes before goal node
                            total_distance = 0.0
                            for i in start_index:(end_index - 1)
                                x1, y1 = stops[i].coord_x, stops[i].coord_y
                                x2, y2 = stops[i + 1].coord_x, stops[i + 1].coord_y
                                total_distance += eucleadian_distance(x1, y1, x2, y2)
                            end
                            distance= total_distance
                            travel_time = (total_distance / bus_velocity) * 60 #travel_time in minutes
                            end_time = node1.start_time + travel_time
                            connections_dict[(origin_node, goal_node)] = Connection(distance, travel_time, end_time)
                        end
                    end
                else
                    if node1.stop_id != node2.stop_id
                        #nodes are on the same bus line but not the same tour
                        distance = eucleadian_distance(node1.coord_x, node1.coord_y, node2.coord_x, node2.coord_y)
                        travel_time = (distance / bus_velocity) * 60 #travel_time in minutes
                        end_time = node1.start_time + travel_time
                        connections_dict[(origin_node, goal_node)] = Connection(distance, travel_time, end_time)
                    end
                end
            else
                # Nodes are on different bus lines
                distance = eucleadian_distance(node1.coord_x, node1.coord_y, node2.coord_x, node2.coord_y)
                travel_time = (distance / bus_velocity) * 60 #travel_time in minutes
                end_time = node1.start_time + travel_time
                connections_dict[(origin_node, goal_node)] = Connection(distance, travel_time, end_time)
            end
        end
    end
end

println("Connections dictionary:")
sort(collect(connections_dict), by = x -> x.first)

Connections dictionary:


4278-element Vector{Pair{Any, Any}}:
 ((0, 0, 0), (1, 1, 1)) => Connection(28.619573721493477, 34.34348846579218, 34.34348846579218)
 ((0, 0, 0), (1, 1, 2)) => Connection(27.520356102347225, 33.02442732281667, 33.02442732281667)
 ((0, 0, 0), (1, 1, 3)) => Connection(21.622673285234647, 25.947207942281576, 25.947207942281576)
 ((0, 0, 0), (1, 1, 4)) => Connection(17.089177862027185, 20.507013434432622, 20.507013434432622)
 ((0, 0, 0), (1, 2, 1)) => Connection(28.619573721493477, 34.34348846579218, 34.34348846579218)
 ((0, 0, 0), (1, 2, 2)) => Connection(27.520356102347225, 33.02442732281667, 33.02442732281667)
 ((0, 0, 0), (1, 2, 3)) => Connection(21.622673285234647, 25.947207942281576, 25.947207942281576)
 ((0, 0, 0), (1, 2, 4)) => Connection(17.089177862027185, 20.507013434432622, 20.507013434432622)
 ((0, 0, 0), (1, 3, 1)) => Connection(28.619573721493477, 34.34348846579218, 34.34348846579218)
 ((0, 0, 0), (1, 3, 2)) => Connection(27.520356102347225, 33.02442732281667, 33.02442732281

In [517]:
a,b,c=(2,1,3)
idx=findfirst(n -> n.bus_line_id==a && n.line_id==b && n.stop_id ==c, nodes)
nodes[idx].start_time

43.85593064973383

In [518]:
connections_dict[((2,1,3),(1,1,1))]

Connection(54.41801172406063, 65.30161406887275, 109.15754471860657)

### Dictionary mit Stopp-IDs je Linie

In [519]:
line_stops_dict = Dict{Int, Vector{Int}}()

for row in eachrow(bus_line_stops)
    l = row.bus_line_id
    stop = row.stop_ids
    # Depot-Zeile überspringen
    if l == 0 && stop == 0
        continue
    end
    if haskey(line_stops_dict, l)
        push!(line_stops_dict[l], stop)
    else
        line_stops_dict[l] = [stop]
    end
end
println("Dictionary of lines with their respective stop-sequences:")
sort(line_stops_dict)

Dictionary of lines with their respective stop-sequences:


OrderedDict{Int64, Vector{Int64}} with 5 entries:
  1 => [1, 2, 3, 4]
  2 => [1, 2, 3]
  3 => [1, 2, 3, 4, 5, 6]
  4 => [1, 2]
  5 => [1, 2, 3, 4, 5]

### Populating set A

In [520]:
function get_connections_which_fulfill_time_constraint_new(lines, line_stops_dict, connections_dict, A_set)
    for (from, to) in sort(collect(keys(connections_dict)))
        # unpack
        a, l1, s1 = from
        c, l2, s2 = to

        # Skip depot node
        if s1 == 0 || s2 == 0
            continue
        end
        # Check if it is a connection from end of one line to start of another line
        if s1 == last(line_stops_dict[a]) && s2 == first(line_stops_dict[c]) && (a != c || l1 != l2)
            #display("Checking for connection $from - $to...")
            idx_origin = findfirst(x -> x.bus_line_id == a && x.line_id == l1 && x.stop_id == first(line_stops_dict[a]), nodes)
            t_start_origin_tour = nodes[idx_origin].start_time
            t_depot_firststop = connections_dict[((0,0,0),(a,l1,first(line_stops_dict[a])))].travel_time
            if t_depot_firststop < t_start_origin_tour
                #display("Origin tour $from (start time $t_start_origin_tour) can be started.")
                t_end_origin_line = connections_dict[((a,l1,first(line_stops_dict[a])),(a,l1,last(line_stops_dict[a])))].end_time
                t_between_lines = connections_dict[((a,l1,last(line_stops_dict[a])),(c,l2,first(line_stops_dict[a])))].travel_time
                idx_goal = findfirst(x -> x.bus_line_id == c && x.line_id == l2 && x.stop_id == first(line_stops_dict[c]), nodes)
                t_start_second_tour = nodes[idx_goal].start_time
                #display("Checking if time constraint holds for connection between tour $from and tour $to...")
                if t_end_origin_line + t_between_lines <= t_start_second_tour
                    #display("Added connection $from-$to, because end of tour $from is $t_end_origin_line and start of tour $to is $t_start_second_tour.")
                    if !((from,to) in A_set)
                        push!(A_set, (from,to))
                    end
                else
                    #display("Time constraint does not hold for connection $from-$to.")
                    continue
                end
            else
                #display("Origin line can not be started in time.")
                continue
            end
        end
    end
end


get_connections_which_fulfill_time_constraint_new (generic function with 1 method)

In [521]:
A_set = []
for i in sort(collect(keys(connections_dict)))
    from, to = i
    #Connection Depot <-> erste Haltestelle wird mit aufgenommen
    if from == (0,0,0) && to[3] == 1
        push!(A_set, i)
    #Connection Depot <-> letzte Haltestelle wird mit aufgenommen
    elseif from[3] !=0 && from[3] == last(line_stops_dict[from[1]]) && to == (0,0,0)
        push!(A_set, i)
    #Connection erste Haltestelle <-> letzte Haltestelle derselben Linie wird mit aufgenommen
    elseif from[3] !=0 && from[3] == first(line_stops_dict[from[1]]) && from[1] == to[1] && to[3] == last(line_stops_dict[to[1]]) && from[2] == to[2]
        push!(A_set, i)
    else
        continue
    end
end

In [522]:
#Connection zwischen den Linien unter der Bedingung "t_end_of_first_line + t_between_lines <= t_start_time_second_line"
get_connections_which_fulfill_time_constraint_new(lines, line_stops_dict, connections_dict, A_set)
full_output = join(string.(A_set), "\n")
print("A_set:\n")
println(full_output)

A_set:
((0, 0, 0), (1, 1, 1))
((0, 0, 0), (1, 2, 1))
((0, 0, 0), (1, 3, 1))
((0, 0, 0), (2, 1, 1))
((0, 0, 0), (2, 2, 1))
((0, 0, 0), (2, 3, 1))
((0, 0, 0), (3, 1, 1))
((0, 0, 0), (3, 2, 1))
((0, 0, 0), (3, 3, 1))
((0, 0, 0), (4, 1, 1))
((0, 0, 0), (4, 2, 1))
((0, 0, 0), (4, 3, 1))
((0, 0, 0), (4, 4, 1))
((0, 0, 0), (5, 1, 1))
((0, 0, 0), (5, 2, 1))
((0, 0, 0), (5, 3, 1))
((0, 0, 0), (5, 4, 1))
((1, 1, 1), (1, 1, 4))
((1, 1, 4), (0, 0, 0))
((1, 2, 1), (1, 2, 4))
((1, 2, 4), (0, 0, 0))
((1, 3, 1), (1, 3, 4))
((1, 3, 4), (0, 0, 0))
((2, 1, 1), (2, 1, 3))
((2, 1, 3), (0, 0, 0))
((2, 2, 1), (2, 2, 3))
((2, 2, 3), (0, 0, 0))
((2, 3, 1), (2, 3, 3))
((2, 3, 3), (0, 0, 0))
((3, 1, 1), (3, 1, 6))
((3, 1, 6), (0, 0, 0))
((3, 2, 1), (3, 2, 6))
((3, 2, 6), (0, 0, 0))
((3, 3, 1), (3, 3, 6))
((3, 3, 6), (0, 0, 0))
((4, 1, 1), (4, 1, 2))
((4, 1, 2), (0, 0, 0))
((4, 2, 1), (4, 2, 2))
((4, 2, 2), (0, 0, 0))
((4, 3, 1), (4, 3, 2))
((4, 3, 2), (0, 0, 0))
((4, 4, 1), (4, 4, 2))
((4, 4, 2), (0, 0, 0))
((5,

### Populating set V

In [523]:
using OrderedCollections

V_set = Set()
push!(V_set, (0, 0, 0))  # Add the depot D

# Loop over all combinations of bus_line_id and line_id in nodes
for (bus_line_id, stop_ids) in line_stops_dict
    # Get all line_ids for this bus_line_id from nodes
    line_ids = unique(n.line_id for n in nodes if n.bus_line_id == bus_line_id)
    
    for line_id in line_ids
        first_stop = first(stop_ids)
        last_stop = last(stop_ids)
        push!(V_set, (bus_line_id, line_id, first_stop))
        push!(V_set, (bus_line_id, line_id, last_stop))
    end
end
V_set = sort(collect(V_set))
full_output = join(string.(V_set), "\n")
println("V_set:")
println(full_output) 

V_set:
(0, 0, 0)
(1, 1, 1)
(1, 1, 4)
(1, 2, 1)
(1, 2, 4)
(1, 3, 1)
(1, 3, 4)
(2, 1, 1)
(2, 1, 3)
(2, 2, 1)
(2, 2, 3)
(2, 3, 1)
(2, 3, 3)
(3, 1, 1)
(3, 1, 6)
(3, 2, 1)
(3, 2, 6)
(3, 3, 1)
(3, 3, 6)
(4, 1, 1)
(4, 1, 2)
(4, 2, 1)
(4, 2, 2)
(4, 3, 1)
(4, 3, 2)
(4, 4, 1)
(4, 4, 2)
(5, 1, 1)
(5, 1, 5)
(5, 2, 1)
(5, 2, 5)
(5, 3, 1)
(5, 3, 5)
(5, 4, 1)
(5, 4, 5)


# Model definition

## Initialize model instance

In [524]:
model= Model(HiGHS.Optimizer)

A JuMP Model
├ solver: HiGHS
├ objective_sense: FEASIBILITY_SENSE
├ num_variables: 0
├ num_constraints: 0
└ Names registered in the model: none

## Initialize variables

In [525]:
@variable(model, x[connection in A_set], Bin)

1-dimensional DenseAxisArray{VariableRef,1,...} with index sets:
    Dimension 1, Any[((0, 0, 0), (1, 1, 1)), ((0, 0, 0), (1, 2, 1)), ((0, 0, 0), (1, 3, 1)), ((0, 0, 0), (2, 1, 1)), ((0, 0, 0), (2, 2, 1)), ((0, 0, 0), (2, 3, 1)), ((0, 0, 0), (3, 1, 1)), ((0, 0, 0), (3, 2, 1)), ((0, 0, 0), (3, 3, 1)), ((0, 0, 0), (4, 1, 1))  …  ((4, 4, 2), (0, 0, 0)), ((5, 1, 1), (5, 1, 5)), ((5, 1, 5), (0, 0, 0)), ((5, 2, 1), (5, 2, 5)), ((5, 2, 5), (0, 0, 0)), ((5, 3, 1), (5, 3, 5)), ((5, 3, 5), (0, 0, 0)), ((5, 4, 1), (5, 4, 5)), ((5, 4, 5), (0, 0, 0)), ((4, 2, 2), (4, 4, 1))]
And data, a 52-element Vector{VariableRef}:
 x[((0, 0, 0), (1, 1, 1))]
 x[((0, 0, 0), (1, 2, 1))]
 x[((0, 0, 0), (1, 3, 1))]
 x[((0, 0, 0), (2, 1, 1))]
 x[((0, 0, 0), (2, 2, 1))]
 x[((0, 0, 0), (2, 3, 1))]
 x[((0, 0, 0), (3, 1, 1))]
 x[((0, 0, 0), (3, 2, 1))]
 x[((0, 0, 0), (3, 3, 1))]
 x[((0, 0, 0), (4, 1, 1))]
 ⋮
 x[((5, 1, 1), (5, 1, 5))]
 x[((5, 1, 5), (0, 0, 0))]
 x[((5, 2, 1), (5, 2, 5))]
 x[((5, 2, 5), (0, 0, 0))]
 x[((5

## Objective function

In [526]:
@objective(model, Min, sum(x[connection] for connection in A_set if connection[1] == (0,0,0)))

x[((0, 0, 0), (1, 1, 1))] + x[((0, 0, 0), (1, 2, 1))] + x[((0, 0, 0), (1, 3, 1))] + x[((0, 0, 0), (2, 1, 1))] + x[((0, 0, 0), (2, 2, 1))] + x[((0, 0, 0), (2, 3, 1))] + x[((0, 0, 0), (3, 1, 1))] + x[((0, 0, 0), (3, 2, 1))] + x[((0, 0, 0), (3, 3, 1))] + x[((0, 0, 0), (4, 1, 1))] + x[((0, 0, 0), (4, 2, 1))] + x[((0, 0, 0), (4, 3, 1))] + x[((0, 0, 0), (4, 4, 1))] + x[((0, 0, 0), (5, 1, 1))] + x[((0, 0, 0), (5, 2, 1))] + x[((0, 0, 0), (5, 3, 1))] + x[((0, 0, 0), (5, 4, 1))]

## Initialize constraints

### Flusserhaltung

In [527]:
#=m = length(keys(line_stops_dict))
println(m)
@variable(model, 0 <= f[(i,j) in A_set] <= m)
@constraint(model, [(i,j) in A_set], f[(i,j)] <= m * x[(i,j)])

# Flow starts at depot
@constraint(model, sum(f[(i,j)] for (i,j) in A_set if i == (0,0)) == m)

# Flow conservation for all other nodes
@constraint(model, [node in V_Set; node != (0,0)],
    sum(f[(i,j)] for (i,j) in A_set if j == node)
  - sum(f[(i,j)] for (i,j) in A_set if i == node)
  == 1
)
=#


@constraint(model, 
    flow_conservation[node in V_set],
    sum(x[connection] for connection in A_set if connection[2] == node) - 
    sum(x[connection] for connection in A_set if connection[1] == node) == 0
)


1-dimensional DenseAxisArray{ConstraintRef{Model, MathOptInterface.ConstraintIndex{MathOptInterface.ScalarAffineFunction{Float64}, MathOptInterface.EqualTo{Float64}}, ScalarShape},1,...} with index sets:
    Dimension 1, Any[(0, 0, 0), (1, 1, 1), (1, 1, 4), (1, 2, 1), (1, 2, 4), (1, 3, 1), (1, 3, 4), (2, 1, 1), (2, 1, 3), (2, 2, 1)  …  (4, 4, 1), (4, 4, 2), (5, 1, 1), (5, 1, 5), (5, 2, 1), (5, 2, 5), (5, 3, 1), (5, 3, 5), (5, 4, 1), (5, 4, 5)]
And data, a 35-element Vector{ConstraintRef{Model, MathOptInterface.ConstraintIndex{MathOptInterface.ScalarAffineFunction{Float64}, MathOptInterface.EqualTo{Float64}}, ScalarShape}}:
 flow_conservation[(0, 0, 0)] : -x[((0, 0, 0), (1, 1, 1))] - x[((0, 0, 0), (1, 2, 1))] - x[((0, 0, 0), (1, 3, 1))] - x[((0, 0, 0), (2, 1, 1))] - x[((0, 0, 0), (2, 2, 1))] - x[((0, 0, 0), (2, 3, 1))] - x[((0, 0, 0), (3, 1, 1))] - x[((0, 0, 0), (3, 2, 1))] - x[((0, 0, 0), (3, 3, 1))] - x[((0, 0, 0), (4, 1, 1))] - x[((0, 0, 0), (4, 2, 1))] - x[((0, 0, 0), (4, 3, 1))] - 

### All bus lines are served entirely

In [528]:
@constraint(model,[connection in A_set; connection[1] != (0,0,0) && connection[1][1] == connection[2][1] && connection[1][2] == connection[2][2]],x[connection] == 1)

JuMP.Containers.SparseAxisArray{ConstraintRef{Model, MathOptInterface.ConstraintIndex{MathOptInterface.ScalarAffineFunction{Float64}, MathOptInterface.EqualTo{Float64}}, ScalarShape}, 1, Tuple{Tuple{Tuple{Int64, Int64, Int64}, Tuple{Int64, Int64, Int64}}}} with 17 entries:
  [((1, 1, 1), (1, 1, 4))]  =  x[((1, 1, 1), (1, 1, 4))] = 1
  [((1, 2, 1), (1, 2, 4))]  =  x[((1, 2, 1), (1, 2, 4))] = 1
  [((1, 3, 1), (1, 3, 4))]  =  x[((1, 3, 1), (1, 3, 4))] = 1
  [((2, 1, 1), (2, 1, 3))]  =  x[((2, 1, 1), (2, 1, 3))] = 1
  [((2, 2, 1), (2, 2, 3))]  =  x[((2, 2, 1), (2, 2, 3))] = 1
  [((2, 3, 1), (2, 3, 3))]  =  x[((2, 3, 1), (2, 3, 3))] = 1
  [((3, 1, 1), (3, 1, 6))]  =  x[((3, 1, 1), (3, 1, 6))] = 1
  [((4, 1, 1), (4, 1, 2))]  =  x[((4, 1, 1), (4, 1, 2))] = 1
                            ⋮
  [((4, 2, 1), (4, 2, 2))]  =  x[((4, 2, 1), (4, 2, 2))] = 1
  [((4, 3, 1), (4, 3, 2))]  =  x[((4, 3, 1), (4, 3, 2))] = 1
  [((4, 4, 1), (4, 4, 2))]  =  x[((4, 4, 1), (4, 4, 2))] = 1
  [((5, 1, 1), (5, 1, 5))

### Other constraints

In [529]:
#@constraint(model, depot_flow_conservation, sum(x[(i,j)] for (i,j) in A_set if i == (0,0)) >= 1)

In [530]:
#@constraint(model, x[((0,0), (5,1))] == 1)

# Solve model

In [531]:
optimize!(model)

Running HiGHS 1.8.0 (git hash: fcfb53414): Copyright (c) 2024 HiGHS under MIT licence terms
Coefficient ranges:
  Matrix [1e+00, 1e+00]
  Cost   [1e+00, 1e+00]
  Bound  [1e+00, 1e+00]
  RHS    [1e+00, 1e+00]
Presolving model
3 rows, 3 cols, 6 nonzeros  0s
0 rows, 0 cols, 0 nonzeros  0s
Presolve: Optimal

        Nodes      |    B&B Tree     |            Objective Bounds              |  Dynamic Constraints |       Work      
     Proc. InQueue |  Leaves   Expl. | BestBound       BestSol              Gap |   Cuts   InLp Confl. | LpIters     Time

         0       0         0   0.00%   16              16                 0.00%        0      0      0         0     0.0s

Solving report
  Status            Optimal
  Primal bound      16
  Dual bound        16
  Gap               0% (tolerance: 0.01%)
  Solution status   feasible
                    16 (objective)
                    0 (bound viol.)
                    0 (int. viol.)
                    0 (row viol.)
  Timing            0.00 (

In [532]:
println("Termination status: ", termination_status(model))
println("Primal status: ", primal_status(model))

if status == MOI.OPTIMAL
    println("Optimale Lösung gefunden!")
    println("Zielfunktionswert: ", objective_value(model))
    active_connections = []
    for connection in A_set 
        if value(x[connection]) > 1e-6
            push!(active_connections, connection)
        end
    end
    sort(active_connections, by = x -> x[1][1][1])
    full_output = join(string.(active_connections), "\n")
    println(full_output)

else
    println("Kein Optimum – Solver-Status: ", status)
end
    

Termination status: OPTIMAL
Primal status: FEASIBLE_POINT
Optimale Lösung gefunden!
Zielfunktionswert: 16.0
((0, 0, 0), (1, 1, 1))
((0, 0, 0), (1, 2, 1))
((0, 0, 0), (1, 3, 1))
((0, 0, 0), (2, 1, 1))
((0, 0, 0), (2, 2, 1))
((0, 0, 0), (2, 3, 1))
((0, 0, 0), (3, 1, 1))
((0, 0, 0), (3, 2, 1))
((0, 0, 0), (3, 3, 1))
((0, 0, 0), (4, 1, 1))
((0, 0, 0), (4, 2, 1))
((0, 0, 0), (4, 3, 1))
((0, 0, 0), (5, 1, 1))
((0, 0, 0), (5, 2, 1))
((0, 0, 0), (5, 3, 1))
((0, 0, 0), (5, 4, 1))
((1, 1, 1), (1, 1, 4))
((1, 1, 4), (0, 0, 0))
((1, 2, 1), (1, 2, 4))
((1, 2, 4), (0, 0, 0))
((1, 3, 1), (1, 3, 4))
((1, 3, 4), (0, 0, 0))
((2, 1, 1), (2, 1, 3))
((2, 1, 3), (0, 0, 0))
((2, 2, 1), (2, 2, 3))
((2, 2, 3), (0, 0, 0))
((2, 3, 1), (2, 3, 3))
((2, 3, 3), (0, 0, 0))
((3, 1, 1), (3, 1, 6))
((3, 1, 6), (0, 0, 0))
((3, 2, 1), (3, 2, 6))
((3, 2, 6), (0, 0, 0))
((3, 3, 1), (3, 3, 6))
((3, 3, 6), (0, 0, 0))
((4, 1, 1), (4, 1, 2))
((4, 1, 2), (0, 0, 0))
((4, 2, 1), (4, 2, 2))
((4, 3, 1), (4, 3, 2))
((4, 3, 2), (0, 0,

# Visualization

In [533]:
# Store full node struct as value for lookup
nodes_dict = Dict{Tuple{Int, Int, Int}, typeof(nodes[1])}()
for connection in active_connections
    origin,goal=connection
    a,b,c = origin
    d,e,f = goal
    node=findfirst(x -> x.bus_line_id == a && x.line_id == b && x.stop_id == c, nodes)
    if !(node in keys(nodes_dict))
        nodes_dict[origin] = nodes[node]
    end
    node=findfirst(x -> x.bus_line_id == d && x.line_id == e && x.stop_id == f, nodes)
    if !(node in keys(nodes_dict))
        nodes_dict[goal] = nodes[node]
    end
end
sort(collect(nodes_dict))

35-element Vector{Pair{Tuple{Int64, Int64, Int64}, @NamedTuple{bus_line_id::Int64, line_id::Int64, stop_id::Int64, coord_x::Float64, coord_y::Float64, start_time::Float64}}}:
 (0, 0, 0) => (bus_line_id = 0, line_id = 0, stop_id = 0, coord_x = 30.0, coord_y = 30.0, start_time = 0.0)
 (1, 1, 1) => (bus_line_id = 1, line_id = 1, stop_id = 1, coord_x = 2.2, coord_y = 36.8, start_time = 20.0)
 (1, 1, 4) => (bus_line_id = 1, line_id = 1, stop_id = 4, coord_x = 20.2, coord_y = 44.0, start_time = 44.338557800136556)
 (1, 2, 1) => (bus_line_id = 1, line_id = 2, stop_id = 1, coord_x = 2.2, coord_y = 36.8, start_time = 40.0)
 (1, 2, 4) => (bus_line_id = 1, line_id = 2, stop_id = 4, coord_x = 20.2, coord_y = 44.0, start_time = 64.33855780013656)
 (1, 3, 1) => (bus_line_id = 1, line_id = 3, stop_id = 1, coord_x = 2.2, coord_y = 36.8, start_time = 60.0)
 (1, 3, 4) => (bus_line_id = 1, line_id = 3, stop_id = 4, coord_x = 20.2, coord_y = 44.0, start_time = 84.33855780013656)
 (2, 1, 1) => (bus_line_id

## Plot

In [534]:
# 1. Prepare node coordinates and hover labels
xs = [n.coord_x for n in nodes]
ys = [n.coord_y for n in nodes]
zs = [n.start_time for n in nodes]

labels = [string("(", n.bus_line_id, ", ", n.line_id, ", ", n.stop_id, ")\n", "t=", round(n.start_time, digits=1)) for n in nodes]

trace_nodes = scatter3d(
    x=xs, y=ys, z=zs,
    mode="markers",
    marker=attr(size=6, color="blue"),
    text=labels,
    hoverinfo="text",
    name="Nodes"
)

# 2. Extract active connections from the model
active_connections = [(i, j) for (i, j) in A_set if value(x[(i, j)]) > 1e-6]

# 3. Generate path-based edges
traces_edges = []

for (start_key, end_key) in active_connections
    start_bus, start_line, _ = start_key
    end_bus, end_line, _ = end_key

    if start_bus == end_bus && start_line == end_line
        # Get all stops for this line
        path_nodes = sort(
            filter(n -> n.bus_line_id == start_bus && n.line_id == start_line, nodes),
            by = n -> n.stop_id
        )

        push!(traces_edges, scatter3d(
            x = [n.coord_x for n in path_nodes],
            y = [n.coord_y for n in path_nodes],
            z = [n.start_time for n in path_nodes],
            mode = "lines",
            line = attr(width=2),
            hoverinfo = "none",
            showlegend = false
        ))
    else
        # Just draw a direct line between tours (e.g., depot → first stop or end → new line)
        x_vals = [nodes_dict[start_key].coord_x, nodes_dict[end_key].coord_x]
        y_vals = [nodes_dict[start_key].coord_y, nodes_dict[end_key].coord_y]
        z_vals = [nodes_dict[start_key].start_time, nodes_dict[end_key].start_time]

        push!(traces_edges, scatter3d(
            x = x_vals, y = y_vals, z = z_vals,
            mode = "lines",
            line = attr(width=2, dash="dot"),
            hoverinfo = "none",
            showlegend = false
        ))
    end
end

# 4. Combine and plot
layout = Layout(
    title="Tour Sequences with Full Paths",
    scene=attr(
        xaxis=attr(title="X"),
        yaxis=attr(title="Y"),
        zaxis=attr(title="Start Time")
    ),
    showlegend=false
)

plot_obj = Plot([trace_nodes; traces_edges...], layout)

# 5. Save plot to interactive HTML file
PlotlyJS.savefig(plot_obj,"/Users/alexanderklaus/Desktop/Masterthesis/Code/output/3d_plot_new.html")


"/Users/alexanderklaus/Desktop/Masterthesis/Code/output/3d_plot_new.html"